# Window 함수
- **데이터 분석하다 궁금해지는 것들**
    - "A라는 유저의 전 주문 수량은?"
    - "B라는 유저가 물건 구입하기전에 몇 번이나 상세보기?"
    - "고객의 구매시점에 총 누적 구매횟수?"
    - "C라는 유저의 이전/다음앱 접속시간이 어떻게?"
    - "랭킹, 누적합"

<br>

## 윈도우 함수 (Window/Analytics Function)
- 데이터의 창문을 통해 보는 것처럼, 일부분씩 보면서 계산
    - 그룹 내의 순위, 누겆 합계, 평균 등

<br>

### 윈도우 함수의 구성

```sql
RANK() OVER (
    PARTITON BY GRADE
    ORDER BY HEIGHT DESC
)
```

- **Window 함수 $\rightarrow$ `RANK()`, `LAG()`, `SUM()`**
- **`OVER` : '~에 걸쳐서', '~에 대해서'**
- **`OVER`절 안의 PARTITION BY, ORDER BY**
- **윈도우 프레임 : 윈도우 함수가 작동하는 범위**

<br>


```sql
함수() OVER (PARTITION BY 파티션컬럼 ORDER BY 정렬컬럼 ROWS BETWEEN A AND B)
```

<br>


| 구분    | 함수          | 설명                         | OVER | PARTITION BY                  | ORDER BY                        | ROWS or RANGE |
| ----- | ----------- | -------------------------- | ---- | ----------------------------- | ------------------------------- | ------------- |
| 집계 분석 | COUNT       | 파티션 범위에서 COUNT             | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 보통 필요 X (단 Frame 설정 시 필요할 수 있음) | 선택            |
| 집계 분석 | AVG         | 파티션 범위에서 AVG               | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 보통 필요 X (단 Frame 설정 시 필요할 수 있음) | 선택            |
| 집계 분석 | SUM         | 파티션 범위에서 SUM               | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 보통 필요 X (단 Frame 설정 시 필요할 수 있음) | 선택            |
| 집계 분석 | MAX/MIN     | 파티션 범위에서 MAX/MIN           | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 보통 필요 X (단 Frame 설정 시 필요할 수 있음) | 선택            |
| 탐색    | LAG         | 파티션 범위에서 ORDER BY 기준 이전 값  | 필수   |보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 필수                              | X             |
| 탐색    | LEAD        | 파티션 범위에서 ORDER BY 기준 이후 값  | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음) | 필수                              | X             |
| 탐색    | FIRST_VALUE | 파티션 범위에서 ORDER BY 기준 첫 값   | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 필수                              | 선택            |
| 탐색    | LAST_VALUE  | 파티션 범위에서 ORDER BY 기준 마지막 값 | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 필수                              | 선택            |
| 번호 지정 | RANK        | 파티션 범위에서 ORDER BY 기준 순위    | 필수   | 보통 필수 (데이터 구조/목적에 따라 없을 수 있음)| 필수                              | X             |
| 번호 지정 | ROW_NUMBER  | 파티션 범위에서 ORDER BY 기준 순서    | 필수   |보통 필수 (데이터 구조/목적에 따라 없을 수 있음) | 필수                              | X             |


<br>

#### 위도우 함수의 종류

- **탐색 함수** : `LEAD`, `LAG`, `FIRST_VALUE`, `LAST_VALUE` 등
- **번호 지정 함수** : `RANK`, `ROW_NUMBER` 등
- **집계 분석 함수** : `AVG`, `COUNT`, `SUM`, `MAX`, `MIN` 등

<br>

<hr>

<br>

## 윈도우 탐색 함수
- **탐색 함수는 행들을 탐색하며, 값을 찾는 함수 $\rightarrow$ 정렬이 항상 필요**
    - 예) 학년별 (`PARTITION BY`)로 줄을 서고, 키가 큰 순으로 정렬 (`ORDER BY`), $\rightarrow$ 정렬한 후, 앞 사람 (`LAG`)의 키는?

<br>

#### `LEAD` : 파티션 내에서 다음 행의 값 반환
- 예) 각 `user_id`별 다음 방문 월

```sql
SELECT
    user_id,
    visit_month,
    LEAD(visit_month, 1) OVER(PARTITION BY user_id ORDER BY visit_month) AS after_visit_month,
    LEAD(visit_month, 2) OVER(PARTITION BY user_id ORDER BY visit_month) AS after_two_visit_month 
FROM TABLE;
```

<img src='img/03-01.png' width=400>


<br>

#### `LAG` : 파티션 내에서 이전 행의 값 반환
- 예) 각 `user_id`별 이전 방문 월

```sql
SELECT
    user_id,
    visit_month,
    LAG(visit_month, 1) OVER(PARTITION BY user_id ORDER BY visit_month) AS before_visit_month,
    LAG(visit_month, 2) OVER(PARTITION BY user_id ORDER BY visit_month) AS before_two_visit_month 
FROM TABLE;
```

<img src='img/03-02.png' width=400>

<br>

#### `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` : 윈도우 프레임의 범위를 명시적으로 지정
- **`UNBOUNDED PRECEDING`** : 파티션의 첫 번째 행부터 시작
- **`UNBOUNDED FOLLOWING`** : 파티션의 마지막 행까지 포함


<br>

#### `FIRST_VALUE` : 파티션 내에서 첫번째 값
- 예) 각 `user_id`별 첫 방문 월

```sql
SELECT
    user_id,
    visit_month,
    FIRST_VALUE(visit_month) OVER(PARTITION BY user_id ORDER BY visit_month ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_visit_month
FROM TABLE;
```

<img src='img/03-03.png' width=400>


<br>

#### `LAST_VALUE` : 파티션 내에서 마지막 값
- 예) 각 `user_id`별 마지막 방문 월

```sql
SELECT
    user_id,
    visit_month,
    LAST_VALUE(visit_month) OVER(PARTITION BY user_id ORDER BY visit_month ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_visit_month
FROM TABLE;
```

<img src='img/03-04.png' width=400>


<br>

<br>

#### `FIRST_VALUE`, `LAST_VALUE`의 NULL 처리
- **일반적인 집계 함수** : `GROUP BY`와 사용되는 `SUM`, `AVG` 집계함수는 **`NULL`을 제외해서 연산**

    **반면, 윈도우 함수의 `FIRST_VALUE`, `LAST_VALUE`에선 기본적으로 `NULL`을 포함해서 연산**

    (파티션 내의 처음/마지막 값이 `NULL`이면 `NULL` 반환)

    $\rightarrow$ **`NULL`을 제외하고 싶다면 `IGNORE NULLS` 추가**

```sql
SELECT
  *,
  FIRST_VALUE(numbers IGNORE NULLS) OVER (ORDER BY date) AS first_num,
  LAST_VALUE(numbers IGNORE NULLS) OVER (ORDER BY date) AS last_num
FROM TABLE;
```

<img src='img/03-05.png' width=300>


<br>

<hr>

<br>

## 윈도우 함수 - 번호 지정 함수

<br>

#### `RANK` : 순위
- 중복이 있으면 공동 순위로 처리하고, 그 다음 값을 패스
  
  (공동 2위 존재시 3등은 없음)
  
<br>

- 예) `product_type`별로 `revenue`의 순위

```sql
SELECT
    *,
    RANK() OVER (PARTITION BY product_type ORDER BY revenue DESC) AS rank
FROM TABLE;
```

<img src='img/03-06.png' width=400>

<br>

#### `ROW_NUMBER` 행의 순서
- 중복이 있으면 랜덤으로 숫자 부여

<br>

- 값이 같아서 공동순위가 나와도, 값을 순차적으로 남기고 싶다면 $\rightarrow$ `ROW_NUMBER`

```sql
SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY product_type ORDER BY revenue DESC) AS row_num
FROM TABLE;
```

<img src='img/03-07.png' width=400>

<br>

#### `ROW_NUMBER`와 `RANK`의 차이
- 중복 값을 어떻게 처리하는 지의 차이
  - `RANK`: 중복이 있으면 공동 순위로 처리하고, 그 다음 값을 패스
    
    (공동 2위 존재시 3등은 없음)

  - `ROW_NUMBER` : 중복이 있으면 랜덤으로 숫자 부여

<br>

#### `ROW_NUMBER` vs `RANK` 선택
- 파티션 내에서 고유한 순서가 필요한 경우 $\rightarrow$ `ROW_NUMBER`

- 파티션 내에서 동일한 값을 가진 행에 대해, 동일한 순서를 부여할 경우 $\rightarrow$ `RANK`
  - 예) 상위 N% 의 그룹화 $\rightarrow$ `RANK`

<br>

<hr>

<br>

## 윈도우 집계 분석 함수

<br>

### 윈도우 함수 vs `GROUP BY`
- **`GROUP BY`가 여러 행의 값을 집계해서 반환하는 반면**
  
  **윈도우 함수는 각각의 행에 값을 계산해서 반환**

<img src='img/03-08.png' width=600>

<br>

#### `AVG` :  평균

```sql
SELECT
    *,
    AVG(revenue) OVER (PARTITION BY product_type) AS avg_revenue
FROM Table;
```

<img src='img/03-09.png' width=300>


<br>

#### `SUM` : 합계

```sql
SELECT 
    *,
    SUM(revenue) OVER (PARTITION BY product_type) AS sum_revenue 
FROM Table;
```

<img src='img/03-10.png' width=300>


<br>

<hr>

<br>

## 윈도우 함수 - 데이터 범위 지정 : Frame 설정
- **데이터의 범위를 제한한 연산의 필요성**
  - 이동 평균 (Moving Average)
  - 특정 시점의 누적 합계
    - 예) 고객별 주문 시점에서 누적 주문량
  - 최근 직전 5개의 평균 주문량

    $\rightarrow$ **Frame을 지정**

<br>

### Frame 설정
- Frame을 설정하는 2가지 방법

1. `ROWS`
- 물리적인 행 수를 기준으로 경계를 지정
  - 예) 이전 행, 이후 3개의 행 
- 더 많이 사용

<br>

2. `RANGE`
- 논리적인 값의 범위를 기준으로 지정
  - 예) 값의 3일전, 3일 후

<br>

#### Frame의 기간 명시

| `UNBOUNDED PRECEDING` | `N PRECEDING` | `CURRENT ROW` | `N FOLLOWING`| `UNBOUNDED FOLLOWING` |
| - | - | - | - | -|
| 그 이전의 모든 시작부터 | 특정 N 앞 전까지 | 현재 ROW | | |
| | | |특정 N 후 까지 | 그 이후의 모든 끝까지|

- **`PRECEDING` : 현재 행 기준으로 이전 행**
- **`CURRENT ROW` : 현재 행**
- **`FOLLOWING` : 현재 행 기준으로 이후 행**
- **`UNBOUNDED` : 처음부터 또는 끝까지 (묶이지 않고 제한되지 않음)**



<br>

- 현재 행과 그 앞뒤 한 행씩 포함

```sql
ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
```

<br>

- 파티션의 처음부터 현재 행을 포함

```sql
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
```

<br>

```sql
SELECT
  *,
  SUM(amount) OVER() AS amount_total, -- 모든 주문량 
  SUM(amount) OVER(ORDER BY order_id) AS cumulative_sum, -- row 시점에 누적 합계
  SUM(amount) OVER(PARTITION BY user_id ORDER BY order_id) AS cumulative_sum_by_user, -- row 시점에 유저별 누적 SUM
  AVG(amount) OVER(ORDER BY order_id ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING) AS last_5_orders_avg_amount -- order_id 기준으로 정렬하고, 직전 5개의 주문의 평균
FROM advanced.orders
ORDER BY order_id
```

<img src='img/03-11.png' width=900>

<br>

<hr>

<br>


## 윈도우 함수 조건 설정 (`Quailify`)
- 21년 4월 이전까지 윈도우 함수를 사용하고 조건 설정을 추가할 시,
  
  서브쿼리를 만들고 `WHERE`절 추가가 필요

  $\rightarrow$ **현재는 `QUALIFY` 추가**

<br>

#### `QUALIFY`
- `WHERE`대신 `QUALIFY`사용하면, 윈도우 함수의 결과에 대해 필터링 가능

```sql
SELECT
    order_id,
    order_date,
    user_id,
    amount,
    SUM(amount) OVER (PARTITION BY user_id) AS amount_total 
FROM advanced.orders
QUALIFY amount_total >=500
```

<br>

<hr>

<br>

## CASE

<br>

#### 사용자별 쿼리를 실행한 총 횟수

```sql
SELECT 
  *,
  COUNT(query_date) OVER (PARTITION BY user) AS total_query_cnt
FROM `advanced.query_logs`
ORDER BY user, query_date;
```

<br>


#### 주차별로 팀 내에서 쿼리를 많이 실행한 수, 주차별로 실행한 수를 활용해 순위
- **순위가 1등인 사람만 필터링**
  
```sql
WITH query_cnt_by_team AS (
  SELECT
    EXTRACT(WEEK FROM query_date) AS week_number,
    team,
    user,
    COUNT(user) AS query_cnt
  FROM advanced.query_logs
  GROUP BY ALL
)

SELECT
  *,
  RANK() OVER(PARTITION BY week_number, team ORDER BY query_cnt DESC) AS rk
FROM query_cnt_by_team
QUALIFY rk = 1
```

<br>

#### 주차별로 쿼리를 실행한 시점 기준 1주 전에 쿼리 실행 수


```sql
WITH query_cnt_by_team AS (
  SELECT
    EXTRACT(WEEK FROM query_date) AS week_number,
    team,
    user,
    COUNT(user) AS query_cnt
  FROM advanced.query_logs
  GROUP BY ALL
)

SELECT
  *,
  LAG(query_cnt, 1) OVER(PARTITION BY user ORDER BY week_number) AS prev_week_query_cnt
FROM query_cnt_by_team
```

<br>

#### 시간의 흐름에 따라, 일자별로 유저가 실행한 누적 쿼리

```sql
SELECT
  *,
  SUM(query_cnt) OVER(PARTITION BY user ORDER BY query_date) AS cumulative_sum,
  SUM(query_cnt) OVER(PARTITION BY user ORDER BY query_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_sum2
FROM (
  SELECT
    query_date,
    team,
    user,
    COUNT(user) AS query_cnt
  FROM advanced.query_logs
  GROUP BY ALL
)
ORDER BY user, query_date;
```

<br>

#### 데이터에서 `NULL`값이라고 되어있는 부분을 바로 이전 날짜의 값으로

```sql
WITH raw_data AS (
  SELECT DATE '2024-05-01' AS date, 15 AS number_of_orders UNION ALL
  SELECT DATE '2024-05-02', 13 UNION ALL
  SELECT DATE '2024-05-03', NULL UNION ALL
  SELECT DATE '2024-05-04', 16 UNION ALL
  SELECT DATE '2024-05-05', NULL UNION ALL
  SELECT DATE '2024-05-06', 18 UNION ALL
  SELECT DATE '2024-05-07', 20 UNION ALL
  SELECT DATE '2024-05-08', NULL UNION ALL
  SELECT DATE '2024-05-09', 13 UNION ALL
  SELECT DATE '2024-05-10', 14 UNION ALL
  SELECT DATE '2024-05-11', NULL UNION ALL
  SELECT DATE '2024-05-12', NULL
)

SELECT
  *,
  CASE WHEN number_of_orders IS NULL THEN LEAD(number_of_orders, 1) OVER (ORDER BY date DESC) ELSE number_of_orders END AS number_of_orders2
FROM raw_data
ORDER BY date;
```

<img src='img/03-12.png' width=400>

<br>



<br>

#### `NULL`을 채운 후, 2일전 ~ 현재 데이터의 평균 (이동 평균)

```sql
WITH raw_data AS (
  ...
),

fill_null AS (
  SELECT
    *,
    CASE WHEN number_of_orders IS NULL THEN LEAD(number_of_orders, 1) OVER (ORDER BY date DESC) ELSE number_of_orders END AS number_of_orders2
  FROM raw_data
  ORDER BY date
)

SELECT 
  *,
  SUM(number_of_orders2) OVER (ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_sum_2,
  AVG(number_of_orders2) OVER (ORDER BY date ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS moving_average_2 
FROM fill_null;
```

<img src='img/03-13.png' width=450>

<br>

#### 커스텀 세션
- 이전 이벤트 로그와 20초가 지나면 새로운 세션으로 간주

```sql
WITH base AS (
  SELECT 
    event_date,
    DATETIME(TIMESTAMP_MICROS(event_timestamp), 'Asia/Seoul') AS event_datetime,
    event_name,
    user_id,
    user_pseudo_id
  FROM `advanced.app_logs`
  WHERE event_date = "2022-08-18"
),

diff_data AS (
  SELECT
    *,
    DATETIME_DIFF(event_datetime, prev_event_datetime, SECOND) AS second_diff
  FROM (
    SELECT
      *,
      LAG(event_datetime, 1) OVER(PARTITION BY user_pseudo_id ORDER BY event_datetime) AS prev_event_datetime
    FROM base
  )
)

SELECT
  *,
  SUM(session_start) OVER(PARTITION BY user_pseudo_id ORDER BY event_datetime) AS session_num
FROM (
  SELECT
    *, 
    CASE
      WHEN prev_event_datetime IS NULL THEN 1
      WHEN second_diff >= 30 THEN 1
      ELSE 0
      END AS session_start
  FROM diff_data
)
ORDER BY event_datetime;
```

<img src='img/03-14.png' width=800>

<br>

<hr>